In [4]:
!conda install --name bananavision --file requirements.txt

Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:url

In [4]:
!setup_environment.sh


In [13]:
import os
import pandas as pd
from sklearn.utils import shuffle
from PIL import Image
import torch
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sklearn'

In [11]:

def create_dataframe_from_images(folders):
    """
    Create a DataFrame by reading images from multiple folders and labeling them with the folder names.

    Args:
        folders (list): List of folder paths to read images from.

    Returns:
        pandas.DataFrame: DataFrame containing image paths and labels.
    """
    data = []
    for folder in folders:
        for filename in os.listdir(folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(folder, filename)
                label = os.path.basename(folder)
                data.append((image_path, label))

    df = pd.DataFrame(data, columns=['image_path', 'label'])
    df = shuffle(df)
    return df

def preprocess_images(df):
    """
    Preprocess the images in the DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame containing image paths and labels.

    Returns:
        tuple: A tuple containing the preprocessed images and corresponding labels.
    """
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    images = []
    labels = []

    for image_path, label in df.values:
        image = Image.open(image_path)
        image = transform(image)
        images.append(image)
        labels.append(label)

    return images, labels

def onehot_encode_labels(labels):
    """
    One-hot encode the labels in the list.

    Args:
        labels (list): List of labels.

    Returns:
        torch.Tensor: One-hot encoded labels.
    """
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoded_labels = encoder.fit_transform(labels).toarray()
    return torch.tensor(encoded_labels)

def train_model(images, labels, num_epochs=10, batch_size=16, num_classes=3):
    """
    Train the ResNet-152 model on the given images and labels.

    Args:
        images (torch.Tensor): Tensor of preprocessed images.
        labels (torch.Tensor): Tensor of one-hot encoded labels.
        num_epochs (int, optional): Number of training epochs. Defaults to 10.
        batch_size (int, optional): Batch size for training. Defaults to 16.
        num_classes(int, optional): Number of classes

    Returns:
        torch.nn.Module: Trained ResNet-152 model.
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = models.resnet152(pretrained=True)  # Use ResNet-152 with pre-trained weights
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)  # Customize the last fully connected layer

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(images, labels), batch_size=batch_size)

    model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {running_loss / 100}")
                running_loss = 0.0

    return model


In [12]:
# Create a DataFrame from the images in the training and validation folders
df_train = create_dataframe_from_images(['data/train/BLACK SIGATOKA_1', 'data/train/FUSARIUM WILT_1','data/train/HEALTHY_1'])
df_validation = create_dataframe_from_images(['data/test/BLACK SIGATOKA_2', 'data/test/FUSARIUM WILT_2','data/test/HEALTHY_2'])


NameError: name 'shuffle' is not defined

In [9]:
display(df_train.head())

,image_path,label
0,data/train/BLACK SIGATOKA_1\Image_870.jpg,BLACK SIGATOKA_1
1,data/train/BLACK SIGATOKA_1\Image_871.jpg,BLACK SIGATOKA_1
2,data/train/BLACK SIGATOKA_1\Image_872.jpg,BLACK SIGATOKA_1
3,data/train/BLACK SIGATOKA_1\Image_873.jpg,BLACK SIGATOKA_1
4,data/train/BLACK SIGATOKA_1\Image_874.jpg,BLACK SIGATOKA_1
